In [3]:
# Nomes: Ana Carolina Poletto, Bárbara Dapper e Louise Zanol northfleet 
import requests
from bs4 import BeautifulSoup

In [ ]:
#1.Coletando múltiplas páginas em loop
import time
#obtendo a página inicial da wikipedia
url = "https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal"
response = requests. get(url)
soup = BeautifulSoup(response.content, "html.parser")
#lista de links que estamos pegando
links = set()  
#refrencia para as paginas que vamos abrir pra pegar os verbetes
prox = 0

while len(links)< 5000:
    #obtendo cada pag
    response = requests. get(url)
    soup = BeautifulSoup(response.content)
    #pego todos os links que são parte do corpo do site, já elimino oq não irei usar
    todos_links = soup.find(id="bodyContent").find_all("a")
    titulo = soup.select(".mw-page-title-main")
    for link in todos_links:
        #checando se uma tag tem determinada propriedade
        #nesse caso estamos vendo se o link é da wikipedia
        if "href" in link.attrs.keys() and link["href"].startswith("/wiki/"):
            #filtrando oq é verbate e oq n é, oq n é verbate tem ":",
            if ":" in link["href"]:
                continue
            response = requests. get("https://pt.wikipedia.org/"+link["href"])
            soup = BeautifulSoup(response.content)
            titulo = soup.select(".mw-page-title-main")

            #verifcando se o título da página nâo está vazio
            if not titulo or not titulo[0].text.strip():
                continue

            #arruma o nome do arquivo
            nome_arquivo = titulo[0].text.strip().replace('"', '').replace('/', '-').replace(" ","_")    
            conteudo = response.content.decode("utf-8")

            #salva no arquivo html
            if conteudo:
                with open(f"{nome_arquivo}.html", "w", encoding="utf-8") as f:
                    f.write(conteudo)
                links.add(nome_arquivo)  

    #convertendo para lista para pegar um próximo link
    url = "https://pt.wikipedia.org/wiki/"+ list(links)[prox]
    prox += 1
    #esperamos meio segundo para fazer uma nova requisção de servidor para não sobrecarregar servido
    time.sleep(0.5)

print(f"Encontrados {len(links)} nesta página")

In [3]:
 #2.Extraindo informações de Infoboxes
import json
links = list(links) 
for link in links:
    url = "https://pt.wikipedia.org/wiki/" + link
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # pegando o título da página para nomear o arquivo
    titulo = soup.find("title").get_text().split(" – ")[0] 

    #localizando a infobox
    box = soup.find_all(attrs={"class": "infobox infobox_v2"})
    #verifica se encontrou uma infobox
    if len(box) > 0:  
        #dicionário para armazenar os pares chave-valor
        info_dict = {}  
        for tag in box[0].find_all("tr"):  
            td_tag = tag.find_all("td") 
            #garante que a linha tem ao menos duas colunas (chave e valor)
            if len(td_tag) >= 2: 
                #usamos join para nâo aparcer as tags do html, somente o texto
                chave = "".join(filho.get_text(strip=True) if filho.name else str(filho) for filho in td_tag[0].children)
                valor = "".join(filho.get_text(strip=True) if filho.name else str(filho) for filho in td_tag[1].children)
                info_dict[chave] = valor  

        #salva os dados em um arquivo json 
        arquivo = f"{titulo}.json"
        with open(arquivo, "w", encoding="utf-8") as json_file:
            json.dump(info_dict, json_file, ensure_ascii=False, indent=4)
